In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time
import os
from data_cleaning import clean_data, new_clean_data
from weather import getCleanWeather
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Lasso
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor, plot_tree
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import pickle

In [7]:
weather = getCleanWeather(r'C:\Users\STH_0\Desktop\Data\Boston')
bluebikes_idle_time = pd.read_csv(r"\Users\STH_0\Desktop\Data\bluebikes_station_location_feature.csv")
bluebikes_idle_time['idle_time'] = pd.to_timedelta(bluebikes_idle_time['idle_time'])
bluebikes_idle_time['idle_time'] = bluebikes_idle_time['idle_time'].dt.total_seconds()
bluebikes_idle_time['idle_time'] = bluebikes_idle_time['idle_time'] / 60
bluebikes_idle_time['start_time'] = pd.to_datetime(bluebikes_idle_time['start_time'])
weather['timestamp'] = pd.to_datetime(weather['timestamp'])
merged_df = pd.merge_asof(bluebikes_idle_time.sort_values('start_time'), weather.sort_values('timestamp'),
                          left_on='start_time', right_on='timestamp',
                          direction='nearest')

In [8]:
merged_df['idle_time_mine'] = merged_df['idle_time_hours'] * 60

In [9]:
merged_df.head()

,station_id_x,station_density,station_id_y,city_center_distance,boston_university_distance,fenway_park_distance,harvard_university_distance,mit_distance,boston_common_distance,boston_public_garden_distance,...,city,timestamp,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature,idle_time_mine
0,67,0.954927,67,2.835790,1.312488,1.311493,2.196322,0.234932,2.246784,1.861094,...,Boston,2017-01-01 00:54:00,4.0,26,Cloudy,1013.5,13.0,0.0,1.0,6.966667
1,36,2.864782,36,1.902449,2.313407,1.674192,3.757982,1.797120,1.114398,0.669948,...,Boston,2017-01-01 00:54:00,4.0,26,Cloudy,1013.5,13.0,0.0,1.0,13.266667
2,36,2.864782,36,1.902449,2.313407,1.674192,3.757982,1.797120,1.114398,0.669948,...,Boston,2017-01-01 00:54:00,4.0,26,Cloudy,1013.5,13.0,0.0,1.0,16.283333
3,46,2.864782,46,2.868917,1.774653,0.989355,3.854331,1.937842,2.078367,1.640679,...,Boston,2017-01-01 00:54:00,4.0,26,Cloudy,1013.5,13.0,0.0,1.0,21.366667
4,10,1.591546,10,4.209433,0.237541,1.001153,2.580400,1.584456,3.514814,3.067487,...,Boston,2017-01-01 00:54:00,4.0,26,Cloudy,1013.5,13.0,0.0,1.0,1.983333


In [10]:
merged_df['hour'] = merged_df['start_time'].dt.hour
merged_df['day'] = merged_df['start_time'].dt.dayofyear
merged_df['month'] = merged_df['start_time'].dt.month

In [11]:
merged_df['idle_time_mine'].describe()

count    2.915398e+06
mean     4.211380e+01
std      7.511708e+01
min      0.000000e+00
25%      5.616667e+00
50%      1.566667e+01
75%      4.135000e+01
max      4.800000e+02
Name: idle_time_mine, dtype: float64

In [12]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2915398 entries, 0 to 2915397
Data columns (total 35 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   station_id_x                   int64         
 1   station_density                float64       
 2   station_id_y                   int64         
 3   city_center_distance           float64       
 4   boston_university_distance     float64       
 5   fenway_park_distance           float64       
 6   harvard_university_distance    float64       
 7   mit_distance                   float64       
 8   boston_common_distance         float64       
 9   boston_public_garden_distance  float64       
 10  newbury_street_distance        float64       
 11  south_station_distance         float64       
 12  north_station_distance         float64       
 13  back_bay_station_distance      float64       
 14  start_time                     datetime64[ns]
 15  end_time       

In [13]:
Y = merged_df['idle_time_mine']
X = merged_df[['station_density',
                    'temperature',
                    'weekday', 
                    'hour', 
                    'day',
                    'month',
                    'cloud_cover',
                    'city_center_distance',
                    'boston_university_distance',
                    'fenway_park_distance',
                    'harvard_university_distance', 
                    'mit_distance', 
                    'boston_common_distance', 
                    'newbury_street_distance', 
                    'south_station_distance', 
                    'north_station_distance', 
                    'back_bay_station_distance',
                    'start_station_id',
                    'end_station_id',
                    'precipitation',
                    'windspeed',
                    'bike_id'
                    ]]




X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

LinearRegression()

In [11]:
y_pred_2d = lin_model.predict(X_test)
print("MAE 2D:", mean_absolute_error(y_test,y_pred_2d))
print("RMSE 2D:", mean_squared_error(y_test,y_pred_2d)**0.5)
print("R2 2D:", r2_score(y_test,y_pred_2d))

MAE 2D: 41.24024170831999
RMSE 2D: 70.67133840185382
R2 2D: 0.11626927472322757


In [12]:
lin_model.score(X_test, y_test)

0.11626927472322757

In [12]:
rf_classifier = DecisionTreeRegressor()
rf_classifier.fit(X_train,y_train)

In [ ]:
rf_classifier.score(X_test, y_test)

0.0607430421118601

In [13]:
rf_classifier = RandomForestRegressor(n_estimators=100, 
                                       bootstrap=True, random_state=42) # we select boostrapp, i.e. we use bagging
rf_classifier.fit(X_train,y_train)

RandomForestRegressor(random_state=42)

In [14]:
rf_classifier.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [15]:
rf_classifier.score(X_test, y_test)

0.5680002592931241

In [16]:
rf_classifier.feature_importances_

array([0.01549528, 0.08333133, 0.05500889, 0.20371258, 0.10599967,
       0.01414232, 0.03815109, 0.01592986, 0.01998361, 0.01100229,
       0.01591904, 0.04494782, 0.01314125, 0.03300858, 0.01298522,
       0.02566931, 0.01013196, 0.02354761, 0.0751831 , 0.01042875,
       0.06747009, 0.10481037])

In [17]:
mse = mean_squared_error(y_test, rf_classifier.predict(X_test))

In [18]:
mse

2441.463095212462

In [23]:
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42, n_estimators=200)
xgb_model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [25]:
xgb_model.score(X_test, y_test)

0.41347904701727767

In [26]:
mean_squared_error(y_test, xgb_model.predict(X_test))

3314.745649923416

In [27]:
baseline_prediction = np.mean(y_train)

# Create an array of the same length as the test data with the baseline prediction
baseline_predictions = np.full_like(y_test, baseline_prediction)
baseline_mse = mean_squared_error(y_test, baseline_predictions)

baseline_mse

5651.53959992439

In [31]:
#save the model

filename = 'finalized_model_random_forest.sav'
pickle.dump(rf_classifier, open(filename, 'wb'))


In [3]:
#read finilized model.sav
filename = 'finalized_model_random_forest.sav'
mdl = pickle.load(open(filename, 'rb'))

In [4]:
mdl.feature_importances_

array([0.01549528, 0.08333133, 0.05500889, 0.20371258, 0.10599967,
       0.01414232, 0.03815109, 0.01592986, 0.01998361, 0.01100229,
       0.01591904, 0.04494782, 0.01314125, 0.03300858, 0.01298522,
       0.02566931, 0.01013196, 0.02354761, 0.0751831 , 0.01042875,
       0.06747009, 0.10481037])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)
mdl.score(X_test, y_test)

0.5680002592931241